In [ ]:
import torch

# probabilistic programming
import pyro 

# MCMC plotting
import arviz as az
import matplotlib.pyplot as plt
from getdist.arviz_wrapper import arviz_to_mcsamples
from getdist import plots

# autoemulate imports
from autoemulate.simulations.epidemic import Epidemic
from autoemulate.core.compare import AutoEmulate
from autoemulate.calibration.bayes import BayesianCalibration
from autoemulate.emulators import GaussianProcess

# suppress warnings in notebook for readability
import os
import warnings

# ignore warnings
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

# random seed for reproducibility
random_seed = 42

In [ ]:
from autoemulate.data.utils import set_random_seed
set_random_seed(random_seed)
pyro.set_rng_seed(random_seed)

In [ ]:
simulator = Epidemic(log_level="error")
x = simulator.sample_inputs(1000)
y, _ = simulator.forward_batch(x)

In [ ]:
transmission_rate = x[:, 0]
recovery_rate = x[:, 1]

plt.scatter(transmission_rate, recovery_rate, c=y, cmap='viridis')
plt.xlabel('Transmission rate (beta)')
plt.ylabel('Recovery rate (gamma)')
plt.colorbar(label="Peak infection rate")
plt.show

In [ ]:
true_beta = 0.3
true_gamma = 0.15 

# simulator expects inputs of shape [1, number of inputs]
params = torch.tensor([true_beta, true_gamma]).view(1, -1)
true_infection_rate = simulator.forward(params)
assert isinstance(true_infection_rate, torch.Tensor)

n_obs = 100
stdev = 0.05
noise = torch.normal(mean=0, std=stdev, size=(n_obs,))
observed_infection_rates = true_infection_rate[0] + noise

observations = {"infection_rate": observed_infection_rates}


In [ ]:
# Run AutoEmulate to find the best GP model
from autoemulate.emulators.gaussian_process.exact import GaussianProcessRBF


ae = AutoEmulate(
    x, 
    y, 
    models=[GaussianProcessRBF],
    model_params={},
    log_level="error", 
)

gp = ae.best_result().model


## Problem set-up: identify an interval excursion set for $f(x)$

The aim for the remainder of this notebook is to explore methods that are able to identify samples $x$ from the interval excursion set.

Mathematically this is:
$$
x \in \mathbb{R}^n, \quad a, b \in \mathbb{R}^m \quad f: \mathbb{R}^n \mapsto \mathbb{R}^m\quad a < f(x) < b
$$

Solving this problem is more general than calculating:
- the level set ($f(x) = c$)
- superlevel set ($f(x) > c$)
- sublevel set ($f(x) < c$)
Howver, each can be formulated such that samples returned can approximate each of these types of level set for crafted values of $a, b$.

In [ ]:
import torch
from torch.distributions import constraints, Transform
from torch.distributions.transforms import AffineTransform, SigmoidTransform

class BoundedDomainTransform(Transform):
    domain = constraints.real
    codomain = constraints.interval(0.0, 1.0)  # will be rescaled
    bijective = True

    def __init__(self, domain_min, domain_max):
        super().__init__()
        self.domain_min = torch.as_tensor(domain_min)
        self.domain_max = torch.as_tensor(domain_max)
        self._sigmoid = SigmoidTransform()
        self._affine = AffineTransform(
            self.domain_min,
            (self.domain_max - self.domain_min)
        )

    def _call(self, x):
        u = self._sigmoid(x)
        return self._affine(u)

    def _inverse(self, y):
        u = (y - self.domain_min) / (self.domain_max - self.domain_min)
        return self._sigmoid.inv(u)

    def log_abs_det_jacobian(self, x, y):
        u = self._sigmoid(x)
        return (
            self._sigmoid.log_abs_det_jacobian(x, u)
            + self._affine.log_abs_det_jacobian(u, y)
        )


In [ ]:
import torch
from torch.special import ndtr
from pyro.distributions import Normal, TransformedDistribution # type: ignore

from autoemulate.core.types import TensorLike


# Problem settings
domain_min = torch.tensor([b[0] for b in simulator.param_bounds])
domain_max = torch.tensor([b[1] for b in simulator.param_bounds])

# y band settings
y_band_low = torch.tensor([0.5])   # lower bound(s) per task
y_band_high = torch.tensor([0.6])  # upper bound(s) per task
d = len(simulator.param_bounds)

MIN_VAR = 1e-12

@torch.no_grad()
def band_prob_from_mu_sigma(mu: torch.Tensor, var_or_cov: torch.Tensor, y1: torch.Tensor, y2: torch.Tensor, aggregate: str = "geomean"):
    """
    Compute per-sample band probability across tasks given GP mean and variance/covariance.

    Parameters
    ----------
    mu : (N, T) or (T,)
    var_or_cov : (N, T) variance per task OR (N, T, T) covariance across tasks
    y1, y2 : (T,) lower/upper bounds per task
    aggregate : 'geomean' | 'sumlog' | 'none'
        - 'geomean': returns geometric mean across tasks (shape N,)
        - 'sumlog': returns sum of log-probs across tasks (shape N,)
        - 'none': returns per-task probabilities (shape N, T)

    Notes
    -----
    - This helper derives per-task standard deviations as sqrt(diag(cov)) when a full covariance
      is provided, or sqrt(variance) when per-task variances are provided.
    """
    if mu.dim() == 1:
        mu = mu.unsqueeze(0)

    # Derive per-task std devs from variance / covariance
    if var_or_cov.dim() == 3:
        var_diag = torch.diagonal(var_or_cov, dim1=-2, dim2=-1).clamp_min(MIN_VAR)
        sigma = var_diag.sqrt()
    else:
        # (N, T) variance or (T,) variance for single sample
        if var_or_cov.dim() == 1:
            var_or_cov = var_or_cov.unsqueeze(0)
        sigma = var_or_cov.clamp_min(MIN_VAR).sqrt()

    # Broadcast bounds to (N, T)
    y1v = y1.view(1, -1)
    y2v = y2.view(1, -1)

    # Stable CDF differences
    a = ((y1v - mu) / sigma).clamp(-30.0, 30.0)
    b = ((y2v - mu) / sigma).clamp(-30.0, 30.0)
    p_task = (ndtr(b.double()) - ndtr(a.double())).clamp_min(1e-12).to(mu.dtype)

    if aggregate == "none":
        return p_task
    log_p = p_task.log()
    if aggregate == "geomean":
        return log_p.mean(dim=-1).exp()
    if aggregate == "sumlog":
        return log_p.sum(dim=-1)
    raise ValueError("aggregate must be one of {'geomean', 'sumlog', 'none'}")


def band_logprob(mu: TensorLike, var: TensorLike, y1: TensorLike, y2: TensorLike, temp=1.0, softness: float | None=None, mix=1.0):
    """
    Multi-task band log-probability with optional soft surrogate and mixing.

    Parameters
    ----------
    mu: TensorLike
        Predicted mean. Shape (N, T) or (T,)
    var: TensorLike
        Predicted variance per task (N, T) or covariance (N, T, T)
    y1: TensorLike
        lower bounds (T,)
    y2: TensorLike
        upper bounds (T,)

    Returns
    -------
    TensorLike
        Scalar per sample (sums over tasks) and then summed over batch here only
        when inputs are 1-sample; callers can keep per-sample if needed by removing .sum().
    """
    # Ensure batch: (N, T)
    if mu.dim() == 1:
        mu = mu.unsqueeze(0)

    # Broadcast bounds to (N, T) for the soft surrogate below
    y1v = y1.view(1, -1)
    y2v = y2.view(1, -1)

    # Exact likelihood across tasks via shared helper (sum of log-probs), with temperature
    log_p_exact = temp * band_prob_from_mu_sigma(mu, var, y1, y2, aggregate="sumlog")

    if softness is None:
        # Return per-sample log-prob; callers can sum if needed
        return log_p_exact.sum()

    # For the soft surrogate we need per-task std; reuse helper path to std
    if var.dim() == 3:
        var_diag = torch.diagonal(var, dim1=-2, dim2=-1).clamp_min(MIN_VAR)
        sigma = var_diag.sqrt()
    else:
        if var.dim() == 1:
            var = var.unsqueeze(0)
        sigma = var.clamp_min(MIN_VAR).sqrt()

    # Soft surrogate per task
    lo = torch.sigmoid((mu - y1v) / (softness * sigma))
    hi = torch.sigmoid((y2v - mu) / (softness * sigma))
    p_soft = (lo * hi).clamp_min(1e-12)
    log_p_soft = p_soft.log().sum(dim=-1)

    # Mixture in log-space per sample
    mix_t = torch.tensor(mix, dtype=mu.dtype, device=mu.device)
    log_p_mix = torch.logaddexp(
        torch.log1p(-mix_t) + log_p_soft,
        torch.log(mix_t) + log_p_exact,
    )
    return log_p_mix.sum()


def make_interval_band_model(temp=1.0, softness=None, mix=1.0):
    def model():
        base = Normal(0., 1.).expand([1, d]).to_event(2)
        transform = BoundedDomainTransform(domain_min, domain_max)
        x_star = pyro.sample("x_star", TransformedDistribution(base, [transform]))
        mu, var = gp.predict_mean_and_variance(x_star)
        assert isinstance(var, TensorLike)
        pyro.factor(
            "band_logp",
            band_logprob(
                mu,
                var,
                y_band_low,
                y_band_high,
                temp=temp,
                softness=softness,
                mix=mix
            ),
        )
    return model

In [ ]:
def plot_samples(samples, num_samples):
    # Ensure correct shape
    samples = samples.reshape(num_samples, -1)
    with torch.no_grad():
        mu_s, var_s = gp.predict_mean_and_variance(samples)
        assert isinstance(var_s, TensorLike)
        # Use shared helper for band probability aggregation, passing var directly
        p_band = band_prob_from_mu_sigma(mu_s, var_s, y_band_low, y_band_high, aggregate="geomean")

    _fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    # Probability in band
    sc1 = ax1.scatter(samples[:, 0].cpu(), samples[:, 1].cpu(), c=p_band.cpu(), cmap="viridis", s=6, alpha=0.7)
    ax1.set_title("Band Probability (agg across tasks)")
    ax1.set_xlabel("x1"); ax1.set_ylabel("x2")
    plt.colorbar(sc1, ax=ax1, label="P[y in band]")

    # Predicted mean
    mu_color = mu_s.mean(dim=-1) if mu_s.dim() > 1 else mu_s
    sc2 = ax2.scatter(samples[:, 0].cpu(), samples[:, 1].cpu(), c=mu_color.cpu(), cmap="viridis", s=6, alpha=0.7)
    ax2.set_title("GP Mean (avg across tasks)")
    ax2.set_xlabel("x1"); ax2.set_ylabel("x2")
    plt.colorbar(sc2, ax=ax2, label="GP Mean")

    plt.tight_layout(); plt.show()

### NUTS (without annealing)

Initially investigate whether posterior samples can be identified with NUTS without adapting density surface during warmup.

In [ ]:
from pyro.infer import NUTS, MCMC

nuts = NUTS(make_interval_band_model(temp=1.0, softness=None, mix=1.0))
mcmc_no_schedule = MCMC(nuts, num_samples=50, warmup_steps=50)
mcmc_no_schedule.run()

In [ ]:
# Plot samples from MCMC without annealing schedule
plot_samples(mcmc_no_schedule.get_samples()["x_star"], mcmc_no_schedule.num_samples)

### NUTS (with annealing)

Initially investigate whether posterior samples can be identified with NUTS at a faster rate through including scheduling during warmup for temperature and softness.

In [ ]:
# Add annealing schedule
class AnnealScheduler:
    def __init__(
            self,
            warmup=500,
            temp_init=0.2,
            temp_final=1.0,
            soft_init=0.5,
            soft_final=0.05,
            mix_init=0.0,
            mix_final=1.0
    ):
        self.warmup = warmup
        self.temp_init = temp_init
        self.temp_final = temp_final
        self.soft_init = soft_init
        self.soft_final = soft_final
        self.mix_init = mix_init
        self.mix_final = mix_final
        self.counter = 0

    def __call__(self):
        frac = min(1.0, self.counter / self.warmup)
        self.counter += 1
        temp = self.temp_init + (self.temp_final - self.temp_init) * frac
        softness = self.soft_init + (self.soft_final - self.soft_init) * frac
        mix = self.mix_init + (self.mix_final - self.mix_init) * frac
        return temp, softness, mix

scheduler = AnnealScheduler(
    warmup=500,
    temp_init=0.01, temp_final=1.0,
    soft_init=0.5, soft_final=0.05,
    mix_init=0.0, mix_final=1.0
)

def tempered_model():
    temp, softness, mix = scheduler()
    return make_interval_band_model(temp=temp, softness=softness, mix=mix)()

In [ ]:
nuts = NUTS(tempered_model)
mcmc_with_annealing = MCMC(nuts, num_samples=100, warmup_steps=100)
mcmc_with_annealing.run()


In [ ]:
plot_samples(mcmc_with_annealing.get_samples()["x_star"], mcmc_with_annealing.num_samples)

## Elliptical slice sampling

Exploring alternative MCMC sampling approaches that are faster to fit.

In [ ]:
# Minimal ESS for Gaussian prior with nonlinear likelihood
# Reference: Murray et al. (2010)
def elliptical_slice(log_likelihood, prior_sample, cur_state, max_trials=100) -> TensorLike:
    nu = prior_sample
    theta = torch.rand(()) * 2 * torch.pi
    theta_min = theta - 2 * torch.pi
    theta_max = theta
    ll_cur = log_likelihood(cur_state)
    ll_threshold = ll_cur + torch.log(torch.rand(())).item()
    for _ in range(max_trials):
        proposal = cur_state * torch.cos(theta) + nu * torch.sin(theta)
        if log_likelihood(proposal) >= ll_threshold:
            return proposal
        if theta < 0:
            theta_min = theta
        else:
            theta_max = theta
        theta = torch.rand(()) * (theta_max - theta_min) + theta_min
    return proposal

@torch.no_grad()
def ess_band_samples(n_draws=2000, burn=200, thin=2, seed=0):
    torch.manual_seed(seed)
    # Use ESS in the whitened space and transform to domain
    base_normal = torch.distributions.Normal(0., 1.).expand([d])
    z = base_normal.sample()  # (d,)
    transform = BoundedDomainTransform(domain_min, domain_max)

    def ll(z_vec):
        x = transform(z_vec.unsqueeze(0))  # (1, d)
        assert isinstance(x, TensorLike)
        mu, var = gp.predict_mean_and_variance(x)
        assert isinstance(var, TensorLike)
        # Multi-task band log-prob per-sample (returns scalar)
        return band_logprob(mu, var, y_band_low, y_band_high)

    samples = []
    total = burn + n_draws * thin
    for t in range(total):
        nu = base_normal.sample()
        z = elliptical_slice(ll, nu, z)
        if t >= burn and ((t - burn) % thin == 0):
            samples.append(transform(z.unsqueeze(0))[0])
    return torch.stack(samples, dim=0)

ess_samps = ess_band_samples(n_draws=2000, burn=200, thin=2, seed=random_seed)

In [ ]:
plot_samples(ess_samps, ess_samps.shape[0])

In [ ]:
import numpy as np

@torch.no_grad()
def ess_diag(samples, max_lag=100):
    """Simple univariate ESS estimate per dim via initial positive sequence."""
    x = samples.cpu().numpy()  # (T, d)
    T, D = x.shape
    ess_dims = []
    for j in range(D):
        s = x[:, j]
        s = (s - s.mean()) / (s.std() + 1e-12)
        rho_sum = 0.0
        for lag in range(1, min(max_lag, T-1)):
            c = np.corrcoef(s[:-lag], s[lag:])[0,1]
            if np.isnan(c):
                break
            if c <= 0:
                break
            rho_sum += 2*c
        ess_j = T / (1.0 + rho_sum)
        ess_dims.append(ess_j)
    return np.array(ess_dims)

ess_per_dim = ess_diag(ess_samps)
print('ESS per dimension (ESS ~ larger is better):', ess_per_dim)
print('Min ESS:', ess_per_dim.min(), 'Median ESS:', np.median(ess_per_dim))

### Importance sampling

An alternative to MCMC is to use importance sampling.

In [ ]:
import torch
from torch.special import ndtr

@torch.no_grad()
def importance_resample(n_candidates=20000, n_keep=2000, m=1, seed=0):
    torch.manual_seed(seed)
    transform = BoundedDomainTransform(domain_min, domain_max)
    # Propose from standard normal in R^{m x d}
    z = torch.randn(
        n_candidates, m, d,
        device=domain_min.device,
        dtype=domain_min.dtype,
    )
    # Map into bounded domain
    x_cand = transform(z)[:, 0, :]  # (n_candidates, d)

    # Weight by band probability under GP (multi-task aware)
    mu, var = gp.predict_mean_and_variance(x_cand)
    assert isinstance(var, TensorLike)
    w = band_prob_from_mu_sigma(mu, var, y_band_low, y_band_high, aggregate="geomean")
    w = (w / w.max()).clamp_min(1e-12)

    # Resample
    idx = torch.multinomial(w, num_samples=n_keep, replacement=True)
    return x_cand[idx]

# Draw and plot
imp_samples = importance_resample(n_candidates=50000, n_keep=5000, seed=123)

In [ ]:
plot_samples(imp_samples, imp_samples.shape[0])

## Sequential Monte Carlo (SMC) with adaptive tempering

SMC is a further alternative to importance sampling that might be expected to scale to higher dimensions slightly better.

Temper the band likelihood from 0 to 1, adaptively controlling steps to hit a target Effective Sample Size (ESS). We resample when ESS falls below the threshold. This converges to the exact target at temperature 1 without gradients.

In [ ]:
@torch.no_grad()
def smc_band(
    n_particles=4000,
    ess_target_frac=0.7,
    max_steps=60,
    move_steps=2,
    rw_step=0.3,
    seed=0,
):
    """
    SMC with adaptive tempering from beta=0 to 1 for the band posterior.
    Includes vectorized random-walk Metropolis rejuvenation in the whitened space.
    Returns: x_final, weights, beta_schedule, ess_history, unique_count
    """
    torch.manual_seed(seed)
    transform = BoundedDomainTransform(domain_min, domain_max)
    device = domain_min.device
    dtype = domain_min.dtype

    # Work in whitened space z ~ N(0, I_d)
    z = torch.randn(n_particles, d, device=device, dtype=dtype)

    def compute_ll(z_batch: torch.Tensor) -> torch.Tensor:
        x = transform(z_batch)  # (N, d)
        assert isinstance(x, TensorLike)
        mu, var = gp.predict_mean_and_variance(x)
        assert isinstance(var, TensorLike)
        return band_prob_from_mu_sigma(mu, var, y_band_low, y_band_high, aggregate="sumlog")  # (N,)

    # Initial log-likelihoods
    ll = compute_ll(z)

    # Initialize weights at beta=0
    logw = torch.zeros(n_particles, device=device, dtype=dtype)
    beta = 0.0
    betas: list[float] = [beta]
    ess_hist: list[float] = []

    def ess_from_logw(lw: torch.Tensor) -> float:
        w = (lw - lw.logsumexp(0)).exp()
        return float(1.0 / (w.pow(2).sum() + 1e-12))

    def systematic_resample(w: torch.Tensor) -> torch.Tensor:
        N = w.numel()
        u0 = torch.rand((), device=w.device, dtype=w.dtype) / N
        cdf = torch.cumsum(w, dim=0)
        thresholds = u0 + torch.arange(N, device=w.device, dtype=w.dtype) / N
        idx = torch.searchsorted(cdf, thresholds)
        return idx.clamp_max(N - 1).long()

    def rejuvenate_rw(z: torch.Tensor, ll: torch.Tensor, beta: float) -> tuple[torch.Tensor, torch.Tensor]:
        # Vectorized RW-MH in z-space with tempered target: prior + beta * ll
        for _ in range(move_steps):
            z_prop = z + rw_step * torch.randn_like(z)
            ll_prop = compute_ll(z_prop)
            dprior = -0.5 * (z_prop.pow(2).sum(dim=1) - z.pow(2).sum(dim=1))
            dlike = beta * (ll_prop - ll)
            log_alpha = dprior + dlike
            accept = torch.log(torch.rand(z.shape[0], device=z.device, dtype=z.dtype)).le(log_alpha)
            if accept.any():
                z = torch.where(accept.unsqueeze(1), z_prop, z)
                ll = torch.where(accept, ll_prop, ll)
        return z, ll

    # Adaptive tempering loop
    for _ in range(max_steps):
        target_ess = ess_target_frac * n_particles
        # Find delta via binary search
        low, high = 0.0, float(1.0 - beta)
        for _ in range(25):
            if high <= 1e-6:
                break
            delta = 0.5 * (low + high)
            lw_try = logw + delta * ll
            ess_try = ess_from_logw(lw_try)
            if ess_try < target_ess:
                high = delta
            else:
                low = delta
        delta = low
        if delta <= 1e-8 and beta < 1.0:
            delta = min(1e-3, 1.0 - beta)
        beta = float(min(1.0, beta + delta))
        logw = logw + delta * ll

        # Normalize and compute ESS
        lw_norm = logw - logw.logsumexp(0)
        w = lw_norm.exp()
        ess = float(1.0 / (w.pow(2).sum() + 1e-12))
        betas.append(beta)
        ess_hist.append(ess)

        need_resample = (ess < target_ess) or (beta >= 1.0 - 1e-6)
        if need_resample:
            idx = systematic_resample(w)
            z = z[idx]
            ll = ll[idx]
            logw.zero_()
            z, ll = rejuvenate_rw(z, ll, beta)

        if beta >= 1.0 - 1e-6:
            break

    x_final = transform(z)
    lw_norm = logw - logw.logsumexp(0)
    w_final = lw_norm.exp()
    unique_count = torch.unique(x_final, dim=0).shape[0]

    return x_final, w_final, torch.tensor(betas), torch.tensor(ess_hist), int(unique_count)

In [ ]:
# Run SMC and plot
smc_particles, smc_w, smc_betas, smc_ess, smc_unique = smc_band(
    n_particles=4000, ess_target_frac=0.6, move_steps=2, rw_step=0.25, seed=123
)


In [ ]:
plot_samples(smc_particles, smc_particles.shape[0])

In [ ]:
# Diagnostic plots
plt.figure(figsize=(5,4))
plt.scatter(smc_particles[:,0].cpu(), smc_particles[:,1].cpu(), s=4, alpha=0.4, c='tab:orange')
plt.title(f'SMC particles (final), unique={smc_unique}/{smc_particles.shape[0]}')
plt.xlabel('x1'); plt.ylabel('x2'); plt.tight_layout()

plt.figure(figsize=(6,3))
plt.plot(smc_betas.cpu().numpy(), '-o', ms=3)
plt.ylabel('beta'); plt.xlabel('step'); plt.title('Temperatures')
plt.tight_layout()

plt.figure(figsize=(6,3))
plt.plot(smc_ess.cpu().numpy(), '-o', ms=3)
plt.ylabel('ESS'); plt.xlabel('step'); plt.title('ESS over steps')
plt.tight_layout()

### Does initializing NUTS with SMC particles speed up inference?

In [ ]:
from pyro.infer import NUTS, MCMC

nuts = NUTS(make_interval_band_model(temp=1.0, softness=None, mix=1.0))
mcmc_smc = MCMC(nuts, num_samples=100, warmup_steps=20, initial_params={"x_star": smc_particles[1:2, :]})
mcmc_smc.run()

In [ ]:
plot_samples(mcmc_smc.get_samples()["x_star"], mcmc_smc.num_samples)

### History matching with multi-task band likelihood

This secion looks at using the current history matching workflow to generate samples from the excursion set.

In [ ]:
from autoemulate.calibration.history_matching import HistoryMatchingWorkflow
import numpy as np

lower = y_band_low.item()
upper = y_band_high.item()
midpoint = 0.5 * (lower + upper)
difference = upper - lower
observations = {"infection_rate": lower + (upper - lower)*torch.rand(100)}

hm = HistoryMatchingWorkflow(
    simulator=simulator,
    result=ae.best_result(),
    observations={"infection_rate": (midpoint, (difference / 4 * 2)**2)}, # 2 * sigma = 0.05
    threshold=1.0, # implausibility threshold in sigma units
    train_x=x,
    train_y=y,
    log_level="error",
)


In [ ]:
# Get samples in NROY space
x_new = simulator.sample_inputs(10000)
mean, variance = gp.predict_mean_and_variance(x_new)
assert isinstance(variance, torch.Tensor)
implausibility = hm.calculate_implausibility(mean, variance)
x_star_nroy = hm.get_nroy(implausibility, x_new)


In [ ]:
plot_samples(x_star_nroy, x_star_nroy.shape[0])

### Compare with a BayesianCalibration approach

This section looks at using the current `BayesianCalibration` approach with a Gaussian-noise observation probabilistic model.


In [ ]:
bc = BayesianCalibration(
    gp, 
    simulator.parameters_range, 
    observations, 
    observation_noise=0.1,
    model_uncertainty=True,
)

Run MCMC using the NUTS sampler. The `BayesianCalibration` class uses Pyro under the hood. Below we use `pyro.set_rng_seed` to ensure reproducibility.


In [ ]:
mcmc_bc = bc.run_mcmc(
    warmup_steps=250, 
    num_samples=500,
    num_chains=2    
)

In [ ]:
# Convert to required format for plotting
x_post_bc = torch.hstack([
    mcmc_bc.get_samples()["beta"].reshape(-1, 1),
    mcmc_bc.get_samples()["gamma"].reshape(-1, 1)
])

In [ ]:
plot_samples(x_post_bc, x_post_bc.shape[0])